<a href="https://colab.research.google.com/github/sachinkun21/Customer_Segmentation-Cohort-Analysis/blob/master/RFM_Analysis(Recency_Frequency_Monetory).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Summary:
1. Introduction
2. Problem Statement

### Introduction
In this Data Tale, You will learn about customer segmentation technique known as RFM segmentation.

Specifically, you will get exposure to 
- Recency, frequency and monetary value
- Create customer segments based on these concepts
- Analyze your results.

#### What is RFM Segmentation?
RFM Segmentation technique is a very popular method of performing Customer Segmentation in Retail Domain.
It is based on Behavioral Customer segmentation approach and focuses following 3 customer behaviour Metrics:
1. **Recency(R)**: This the measurement of how recent the customers last purchase was.
2. **Frequency(F)**: This measurement is for how many times the customer has purchased in the specific time period for which data was collected
3. **Monetary(M)**: Measures the total amount the customer has spent during the data collection period.



We will calculate these 3 metrics for each customer and then use them to group customers into predefined specfic categories based on Business requirement or Demand, such as: High, Low , Medium

#### Grouping criteria using RFM values

In Order to group our customers based on any business metric, we need to decide on number of groups as well as threshold or limits of business metric for each group.

There are many ways to decide this. Given below are some of the popular ways to achieve this:

- Equal Percentiles: We can divide our customers into groups of Equal size i.e percentiles and assign them corresponding labels. For example, if we select 4 groups than each group will have 25% of total customers.

- Pareto 80:20 cut: Pareto Principle states that 80 percent of  outcomes come from 20 percent of inputs. This can extended to E-Commerce domain as well with assumption that 80% of revenue is generated by top 20 percentile of Customers. This 80:20 rule can also be used as a criteria to segment customers into two groups the top 20 or high value customers and remaining 80 i.e Low Value Customers

- Custom-based on business knowledge: We can also use existing business knowledge or consult with Subject matter experts(SME's) to decide thresholds and number of segments to be created.

### Importing Libraries and transaction Dataset from Drive into Dataframe

In [0]:
import numpy as np
import pandas as pd

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
ls /content/drive/'My Drive'/DataScience

Employee.csv  Employee_HR.csv  transaction.csv


In [13]:
# linear algebra
import numpy as np
# data processing
import pandas as pd

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Reading "turnover.csv" file and saving it in a DataFrame
df = pd.read_csv("/content/drive/My Drive/DataScience/transaction.csv", index_col='Unnamed: 0')

# printing top 5 rows of DataFrame
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom
